In [ ]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")
import math
import os
import datetime
from pandas import DataFrame
import itertools
from bs4 import BeautifulSoup
import urllib.request
import requests
import random
import torch
import xmltodict
import xml.etree.ElementTree as ET
import xml
from xml.etree.ElementTree import parse
from xml.parsers import expat
from xml.dom.minidom import parse, parseString
import psycopg2
import PyPDF2
import pikepdf
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import re
import spacy

# **LOAD BILL XML DATA to LOCAL DIRECTORY FILES**

In [ ]:
def createfiledirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
    return directory

In [ ]:
def creaturllist(searchpdf, tplist, ext, addcheck):
    typenapd = pd.DataFrame(columns=['Name', 'Link', 'OfficialTitle', 'BillStage', 'Date',  'LegisType', 'CommitteeName', 
                                    'SponsorLastName', 'SponsorFirstName', 'SponsorState',  'CoSponsorLastName', 'CoSponsorFirstName', 'CoSponsorState'])
    for page in searchpdf.pages:
        for annots in page.get("/Annots"):
            url=str(annots.get("/A").get("/URI"))
            if url not in typenapd['Link'].values and url.find(tplist)!=-1:
                typena=url.split(tplist)[1].split(ext)[0]
                if isinstance(addcheck, str):
                    typenapd=typenapd.append({'Name':typena, 'Link':url}, ignore_index=True)
                else:
                    if typena not in addcheck['Name'].values:
                        typenapd=typenapd.append({'Name':typena, 'Link':url}, ignore_index=True)
    return typenapd

def importfiles(filepd, filedir, typestr):   
    for ind in filepd.index:
        name=filepd.loc[ind, 'Name']
        link=filepd.loc[ind, 'Link']
        response = requests.get(link)
        file=filedir+ '/'+name+typestr
        with open(filedir + '/'+name+typestr, 'wb') as file:
            file.write(response.content)
            file.close()
    return 

# **LOAD & TRANSFORM COMMITTEE INFO FROM TEXT FILE**

In [ ]:
def findchair(com): 
    com=com.lstrip(' ')
    if 'vice' in com:
        com=com.split(', vi')[0]
        chair='vice chair'
    elif 'chair' in com or 'chiar' in com:
        com=com.split(', ch')[0]
        chair='chair'
    else:
        chair=''
    return com, chair

def lastnamefix(lastname):
    if "'" in lastname:
        lastname=lastname.split("``")[0].rstrip(' ')
    return lastname

def longnames(name):
    if ',' in name:
        namelist=name.replace('.', '').split(',')
        name=namelist[0].replace('\n','').lstrip(' ')
        if 'of' not in namelist[1]:
            lastname=namelist[1].lstrip(' ')
            state=namelist[2].replace('\n','').split('of ')[1].lstrip(' ')
        else:
            lastname=namelist[1].replace('\n','').split(' of ')[0].lstrip(' ')
            state=namelist[1].replace('\n','').split(' of ')[1].lstrip(' ')
        lastname=lastnamefix(lastname)
    else:
        lastname=''
        state='' 
    return name, lastname, state

In [ ]:
def loadrepdatafromxl(commlist):
    reppd=pd.DataFrame(columns=['Representative Name','Last Name','State', 'Committee', 'Role'])
    for nc in commlist.split('\n \n'):
        if len(nc.replace('.,','').split('...'))>1:
            name, lastname, state = longnames(list(filter(None, nc.replace('.,','').split('...')))[0].replace('\n','').lstrip(' '))
            replist=list(filter(None,list(filter(None,nc.replace('.,','').split('...')))[1].replace('  ', '').replace('\n', '').split('.')))
            for com in replist:
                com, chair=findchair(com)
                if 'Page' not in str(com):
                    reppd=reppd.append({'Representative Name': name,'Last Name':lastname,'State':state, 'Committee': com, 'Role':chair}, ignore_index=True)
        elif 'Page' not in str(nc.replace('.,','').split('...')):
            replist=list(filter(None,nc.replace('.,','').split('...')[0].replace('  ', '').replace('\n','').split('.')))
            if 'of' not in replist[0]:
                for com in replist:
                    com, chair=findchair(com)
                    reppd=reppd.append({'Representative Name': name,'Last Name':lastname,'State':state, 'Committee': com, 'Role':chair}, ignore_index=True)
            else:
                com, chair=findchair(replist[1].lstrip(' '))
                reppd=reppd.append({'Representative Name': replist[0].split(',')[0],'Last Name':lastnamefix(replist[0].split(',')[1].split(' of ')[0].lstrip(' ')),
                                    'State':replist[0].split(',')[1].split(' of ')[1],'Committee': com, 'Role':chair}, ignore_index=True)
    return reppd

# **LOAD & TRANSFORM BILL XML DATA FROM LOCAL FILE**

In [ ]:
def iterroot(root, rootstr, elements):
    detlist=[]
    if rootstr in elements:
        for var in root.iter(rootstr):
            detlist.append(var.text)

    if len(detlist)==0:
        detlist=''
    elif len(detlist)==1:
        detlist=detlist[0]
    return detlist

In [ ]:
def breaknames(name, xmlnames, pdstr):
    if name == '':
        first=''
        last=''
        state=''
    else:
        if 'of ' in name:
            state=name.split('of ')[1]
            fullname=name.split(' of ')[0].split(' ')[1:]
        else:
            state=''
            fullname=name.split(' ')[1:]
        if len(fullname)==3:
            first=fullname[0]+ ' ' + fullname[1][0]
            last=fullname[2]
        elif len(fullname)==2:
            first=fullname[0]
            last=fullname[1]
        elif len(fullname)==4:
            first=fullname[0]+ ' ' + fullname[1][0]
            last=fullname[2] + ' '+ fullname[3]
        elif len(fullname)==1:
            first=''
            last=fullname[0]
    return first, last, state

In [ ]:
def parsexml(xmlnames):
    xmlnames=xmlnames.set_index('Name')
    xmlnames['Date'].astype('object')
    xmlnames['CommitteeName'].astype('object')
    xmlnames['SponsorLastName'].astype('object')
    xmlnames['SponsorFirstName'].astype('object')
    xmlnames['SponsorState'].astype('object')
    xmlnames['CoSponsorLastName'].astype('object')
    xmlnames['CoSponsorFirstName'].astype('object')
    xmlnames['CoSponsorState'].astype('object')

    headerpd=pd.DataFrame(columns=['BillID', 'BillName', 'Header', 'Key', 'Text'])
    billid=0
    for billname in xmlnames.index:
        filename=directoryxml + '/' + billname + '.xml'
        tree = ET.parse(filename)
        root = tree.getroot()
        elements=[elem.tag for elem in root.iter()]
        xmlnames.loc[billname, 'OfficialTitle']=iterroot(root, 'official-title', elements)
        xmlnames.loc[billname, 'LegisType']=iterroot(root,'legis-type', elements)
        xmlnames.at[billname, 'Date']=iterroot(root,'action-date', elements)
        xmlnames.at[billname, 'CommitteeName']=iterroot(root,'committee-name',elements)
        spon=iterroot(root,'sponsor',elements)
        cospon=iterroot(root,'cosponsor',elements)
        cfirst=[]
        clast=[]
        cstate=[]
        first=[]
        last=[]
        state=[]
        if '[' not in str(cospon) and str(cospon)!='':
            cfirst, clast, cstate= breaknames(cospon, xmlnames, 'CoSponsor')
        elif str(cospon)!='':
            for co in cospon:
                cfirstn, clastn, cstaten= breaknames(co, xmlnames, 'CoSponsor')
                cfirst.append(cfirstn)
                clast.append(clastn)
                cstate.append(cstaten)
        if '[' not in str(spon) and str(spon)!='':
            first, last, state= breaknames(spon, xmlnames, 'Sponsor')
        elif str(spon)!='':
            for sp in spon:
                firstn, lastn, staten= breaknames(sp, xmlnames, 'CoSponsor')
                first.append(firstn)
                last.append(lastn)
                state.append(staten)

        xmlnames.at[billname, 'SponsorLastName']=last
        xmlnames.at[billname, 'SponsorFirstName']=first
        xmlnames.at[billname, 'SponsorState']=state
        xmlnames.at[billname, 'CoSponsorLastName']=clast
        xmlnames.at[billname, 'CoSponsorFirstName']=cfirst
        xmlnames.at[billname, 'CoSponsorState']=cstate

        for var in root.iter('bill'):
            stage=var.attrib['bill-stage']
        xmlnames.loc[billname, 'BillStage']=stage
        lasttag=''
        for var in root.iter('*'):
            tag=var.tag
            text=var.text
            if tag =='header':
                lasttag='header'
                header=text
                headerpd=headerpd.append({'BillID':billid, 'BillName': billname, 'Header':header, 'Key':tag, 'Text':text}, ignore_index=True)
            elif tag=='paragraph':
                lasttag='paragraph'
            elif lasttag=='header' and tag in ['text', 'quote', 'short-title', 'external-xref', 'quoted-block', 'term']:
                 if str(text)!='None':
                    headerpd=headerpd.append({'BillID':billid, 'BillName': billname, 'Header':header, 'Key':tag, 'Text':text}, ignore_index=True)
        billid=billid+1    
    return xmlnames, headerpd

In [ ]:
def combinebillheadertext(headerpd):
    headercpd=headerpd.groupby(['BillID','Header'])['Text'].apply('; '.join).reset_index()
    headercpd2=pd.DataFrame(columns=['BillID', 'Header', 'Order', 'Text'])
    billid=headerpd.loc[0, 'BillID']
    countheader=0
    for ind in headerpd.index:
        if headerpd.loc[ind, 'Key']=='header':
            header=headerpd.loc[ind, 'Text']
            if isinstance(headerpd.loc[ind, 'Text'], (str, int, float)):
                text=str(headerpd.loc[ind, 'Text']).replace('Äî','-')
            countheader=countheader+1
        if headerpd.loc[ind, 'Key']!='header':
            if isinstance(headerpd.loc[ind, 'Text'], (str, int, float)):
                if headerpd.loc[ind-1, 'Key']=='header':
                    text=text+': '+str(headerpd.loc[ind, 'Text']).replace('Äî','-')
                else:
                    text=text+str(headerpd.loc[ind, 'Text']).replace('Äî','-')
        if ind!=len(headerpd)-1:
            if headerpd.loc[ind+1, 'Key']=='header':
                headercpd2=headercpd2.append({'BillID':billid, 'Header':header, 'Order':countheader, 'Text':text}, ignore_index=True)
            if headerpd.loc[ind, 'BillID']!=headerpd.loc[ind+1, 'BillID']:
                billid=headerpd.loc[ind+1, 'BillID']
                countheader=0
        else:
             headercpd2=headercpd2.append({'BillID':billid, 'Header':header, 'Order':countheader, 'Text':text}, ignore_index=True)
    return headercpd2

# **DETERMINE KEY ATTRIBUTES & RELATIONS FROM BILL TEXT DATA**

In [ ]:
def findkeyword(headers):
    tokenpd=pd.DataFrame(columns=['Header Index', 'Text', 'De-Conjugate', 'POS Descr', 'POS', 'POS Text', 'Caps'])
    wordpd=pd.DataFrame(columns=['BillID', 'HeaderID', 'HeaderText'])
    for ind in headers.index:
        billid=headers.loc[ind, 'BillID']
        text=headers.loc[ind,'Text'].replace('-', ' ').replace('     ', '')

        nlp = spacy.load("en_core_web_sm")
        doc = nlp(text) 
        if str(doc) not in ['Congress makes following findings:', 'It is sense Senate that—']:
            tokenpd=pd.DataFrame(columns=['Text', 'De-Conjugate', 'POS Descr', 'POS', 'POS Text', 'Caps'])
            for token in doc:
                if token.pos_ not in ['DET', 'ADP', 'CCONJ', 'PUNCT', 'SPACE'] or   str(token.text).lower() in ['no', 'some', 'all','every','each', 'any', 'after', 'before'] :
                    if token.pos_ not in ['AUX'] and  token.tag_ not in ['VB'] :
                        tokenpd=tokenpd.append({ 'Text':token.text, 'De-Conjugate': token.lemma_, 'POS Descr':token.pos_,
                                                    'POS':token.tag_, 'POS Text':token.dep_, 'Caps':token.shape_},ignore_index=True)
                        print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
            tokenat=tokenpd
            plist=[]
            propcomb=''
            for indtok in tokenat.index:
    
                if  tokenat.loc[indtok, 'POS Descr']=='PROPN' and tokenat.loc[indtok, 'POS Text']=='punct':
                    tokenat.loc[indtok, 'POS Descr']='Wrong'
                elif tokenat.loc[indtok, 'POS Descr']=='PROPN' and str(tokenat.loc[indtok, 'Text'])[0]!= str(tokenat.loc[indtok, 'Text'])[0].upper():
                    tokenat.loc[indtok, 'POS Descr']='Wrong'
            
                if tokenat.loc[indtok, 'POS Descr']=='PROPN' and indtok==0:
                    propcomb=tokenat.loc[indtok, 'Text']
                elif tokenat.loc[indtok, 'POS Descr']=='PROPN'  and tokenat.loc[indtok, 'POS Text'] in ['appos', 'conj','nmod']:
                    propcomb=propcomb + ' ' + tokenat.loc[indtok, 'Text']
                elif tokenat.loc[indtok, 'POS Descr']=='PROPN' and tokenat.loc[indtok-1, 'POS Descr']=='PROPN'  and tokenat.loc[indtok, 'POS Text'] not in ['nmod','appos', 'conj'] and tokenat.loc[indtok-1, 'POS Text'] not in ['dobj','pobj',  'nsubj']:
                    propcomb=propcomb + ' ' + tokenat.loc[indtok, 'Text']
                elif tokenat.loc[indtok, 'POS Descr']=='PROPN' and tokenat.loc[indtok-1, 'POS Descr']=='PROPN' and tokenat.loc[indtok-1, 'POS Text'] in ['dobj','pobj', 'nsubj']:
                    propcomb= tokenat.loc[indtok, 'Text']
                elif tokenat.loc[indtok, 'POS Descr']=='PROPN' and tokenat.loc[indtok-1, 'POS Descr']!='PROPN':
                    propcomb= tokenat.loc[indtok, 'Text']
                
                if indtok<=len(tokenat.index)-2:
                    if tokenat.loc[indtok+1, 'POS Descr']!='PROPN' :
                        if propcomb !='' and propcomb not in plist and str(propcomb)[0]==str(propcomb)[0].upper():
                            plist.append(propcomb)
                            propcomb=''
                        elif propcomb in plist:
                            propcomb=''
                    elif tokenat.loc[indtok+1, 'POS Descr']=='PROPN' and tokenat.loc[indtok, 'POS Text'] in ['pobj', 'nsubj', 'dobj'] and tokenat.loc[indtok+1, 'POS Text'] not in ['appos','conj','nmod']:
                        if propcomb !='' and propcomb not in plist and str(propcomb)[0]==str(propcomb)[0].upper():
                            plist.append(propcomb)
                            propcomb=''
                        elif propcomb in plist:
                            propcomb=''
                else:
                    if tokenat.loc[indtok, 'POS Descr']=='PROPN' and propcomb not in plist and str(propcomb)[0]==str(propcomb)[0].upper():
                        plist.append(propcomb) 
                        propcomb=''
                    elif tokenat.loc[indtok, 'POS Descr']=='PROPN' and propcomb in plist:
                        propcomb=''

            if len(tokenat[tokenat['POS Descr']=='PROPN'])!=0 and '[]' not in str(plist):
                for word in plist:
                    if 'U.S.C' not in word:
                        print(word)
                        wordpd=wordpd.append({'BillID': billid, 'HeaderID': ind, 'HeaderText':word}, ignore_index=True)
                    
        else:
            if str(doc) in ['Congress makes following findings:']:
                wordpd=wordpd.append({'BillID': billid,'HeaderID': ind,  'HeaderText':'Congress Findings'}, ignore_index=True)
            if str(doc) in ['It is sense Senate that—']:
                wordpd=wordpd.append({'BillID': billid,'HeaderID': ind,  'HeaderText':'Senate Header'}, ignore_index=True)
    return tokenpd, wordpd

In [ ]:
def findkeywordlabel(headers):
    nlp = spacy.load("en_core_web_sm")
    wordlabelpd=pd.DataFrame(columns=['BillID', 'HeaderID', 'HeaderText', 'CategoryLabel'])
    for ind in headers.index:
        billid=headers.loc[ind, 'BillID']
        doc = nlp(headers.loc[ind,'Text'])
        for ent in doc.ents:
            print(ent.text, ent.start_char, ent.end_char, ent.label_)
            wordlabelpd=wordlabelpd.append({'BillID':headers.loc[ind, 'BillID'], 'HeaderID':ind, 'HeaderText': ent.text, 'CategoryLabel': ent.label}, ignore_index=True)
    return wordlabelpd

In [ ]:
def createuniquewordlist(wordlabelpd):
    uniquewordpd=pd.DataFrame(columns=['WordText', 'CategoryLabel'])
    wordlist=[]
    for ind in wordlabelpd.index:
        if wordlabelpd.loc[ind, 'HeaderText'].lower() not in wordlist:
            wordlist.append(wordlabelpd.loc[ind, 'HeaderText'].lower())
            uniquewordpd=uniquewordpd.append({'WordText': wordlabelpd.loc[ind, 'HeaderText'], 'CategoryLabel': wordlabelpd.loc[ind, 'CategoryLabel']}, ignore_index=True)
    
    return uniquewordpd

In [ ]:
def addextrapd(wordlabelpd, wordpd):
    wordlabelpdco=wordlabelpd.groupby(['BillID', 'HeaderID'])['HeaderText'].apply(list).reset_index()
    for word in wordpd.index:
        wordlabelpdcose=wordlabelpdco[wordlabelpdco['BillID']==wordpd.loc[word, 'BillID']]
        wordlabelpdcose=wordlabelpdcose[wordlabelpdcose['HeaderID']==wordpd.loc[word, 'HeaderID']]
        if len(wordlabelpdcose)==1:
            if wordpd.loc[word, 'HeaderText'] not in  wordlabelpdcose.values[0]:
                  wordlabelpd=wordlabelpd.append({'BillID':wordpd.loc[ind, 'BillID'], 'HeaderID':wordpd.loc[word, 'HeaderID'], 'HeaderText': wordpd.loc[word, 'HeaderText']  , 
                                                  'CategoryLabel':  'PropNounOther'}, ignore_index=True)
    return wordlabelpd

# **EXECUTE DATA IMPORT & TRANSFORM FUNCTIONS**

In [ ]:
username='joythompson'
dbname='postgres'
repinfo=pd.read_csv('/Users/' +username+'/Desktop/DSE_203_Final_Project_JoyT/us-115th-congress-members-fixed.csv')
commlist= open('/Users/' +username+'/Desktop/DSE_203_Final_Project_JoyT/Committees.txt','r').read()
searchpdf = pikepdf.Pdf.open("search.pdf")   
directoryxml=createfiledirectory( '/Users/joythompson/Desktop/DSE_203_Final_Project_JoyT/DSE_project_xml')
directorytext=createfiledirectory( '/Users/joythompson/Desktop/DSE_203_Final_Project_JoyT/DSE_project_text')

In [ ]:
xmlnames=creaturllist(searchpdf, '/xml/', '.xml', 'None')
textnames=creaturllist(searchpdf, '/html/', '.htm', xmlnames)
#importfiles(xmlnames, directoryxml, '.xml')
#importfiles(textnames, directorytext, '.text')

In [ ]:
reppd= loadrepdatafromxl(commlist)
xmlnames, headerpd=parsexml(xmlnames)
headerorderpd=combinebillheadertext(headerpd)

In [ ]:
wordlabelpd=findkeywordlabel(headerorderpd)

In [ ]:
tokenpd, wordpd=findkeyword(headerorderpd)

In [ ]:
uniquewordpd=createuniquewordlist(wordlabelpd)

In [ ]:
wordlabelpd.to_csv('word label.csv')
uniquewordpd.to_csv('uniqueword.csv')

In [ ]:
 wordlabelpd=addextrapd(wordlabelpd, wordpd)

In [ ]:
LOAD CSV WITH HEADERS FROM "https://github.com/JoyT10294/DSE203_FinalProj/blob/main/polirepsdata.csv" AS repdata
CREATE (politician:Politician {id: repdata.representativeid, lastname: repdata.lastname, firstname: repdata.firstname})
CREATE (state:State {id: repdata.statecode, name: repdata.statelabel})

CREATE (politician)-[:REPRESENTS {district:repdata.district}]->(state)
CREATE (party:Party {name: repdata.party})
CREATE (politician)-[:MEMBER_OF]->(party)
CREATE (chamber:Chamber {name: repdata.chamber})
CREATE (politician)-[:SERVES_IN]->(chamber)


# **DATA LOAD TO POSTGRES**

***REPRESENTATIVES***

In [ ]:
conn = psycopg2.connect(database=dbname, user=username, password='', host='127.0.0.1', port='5432')
conn.autocommit = True
cursor = conn.cursor()
queryschema='Create schema dse203finalproject'
#cursor.execute(queryschema)
queryrep='CREATE TABLE dse203finalproject.StateReps (RepresentativeID SERIAL PRIMARY KEY NOT NULL, LastName varchar(255), \
    FirstName varchar(255), StateLabel varchar(255), StateCode int, District int, Chamber varchar(255), Party varchar(255))'
#cursor.execute(queryrep)
countind=0
for ind in repinfo.index:
    first=repinfo.loc[ind, 'First Name'][1:].replace("'", '').replace('.','')
    if len(first.split(' ')[0])==1:
        first=first.split(' ')[1]+' '+first.split(' ')[0]
    queryinsert="INSERT INTO dse203finalproject.StateReps  VALUES ("+str(countind)+", "+str("'"+repinfo.loc[ind, 'Last Name'].replace("'", '')+"'")+ ", "+ \
        str("'"+first+"'")+ ", "+ str("'"+repinfo.loc[ind, 'State Label']+"'")+ ", "+str(repinfo.loc[ind, 'State Code'])+ ", "+str(repinfo.loc[ind, 'District'])+ ", " \
        +str("'"+repinfo.loc[ind, 'Chamber']+"'") +", "+str("'"+repinfo.loc[ind, 'Party']+"'")+ ")" 
    #cursor.execute(queryinsert)
    countind=countind+1

***COMMIITTEES***

In [ ]:
querycomm='CREATE TABLE dse203finalproject.Committees (RepCommID SERIAL PRIMARY KEY NOT NULL, RepresentativeName varchar(255), \
    FirstName varchar(255), State varchar(255), CommitteeName varchar(255), CommitteeRole varchar(255))'
#cursor.execute(querycomm)
countind=0
for ind in reppd.index:
    queryinsert="INSERT INTO dse203finalproject.Committees  VALUES ("+str(countind)+", "+str("'"+reppd.loc[ind, 'Representative Name'].replace("'", '')+"'")+ \
        ", "+ str("'"+reppd.loc[ind, 'Last Name'].replace("'", '')+"'")+ ", " +str("'"+reppd.loc[ind, 'State']+"'")+ ", "+ \
        str("'"+reppd.loc[ind, 'Committee'].replace("'", '')+"'")+ ", "+ str("'"+reppd.loc[ind, 'Role']+"'")+  ")" 
    #cursor.execute(queryinsert)
    countind=countind+1

***COMMIITTEES Referencing REPRESENTATIVE ID***

In [ ]:
querycomm2='CREATE TABLE dse203finalproject.CommitteesbyID(SeatID SERIAL PRIMARY KEY NOT NULL, RepresentativeID INT references \
    dse203finalproject.StateReps (RepresentativeID) NOT NULL, CommitteeName varchar(255), Chair varchar(255))'    
#cursor.execute(querycomm2)
querycomm2in='INSERT INTO dse203finalproject.CommitteesbyID (RepresentativeID, CommitteeName, Chair) \
Select StateReps.RepresentativeID as "RepresentativeID", Committees.CommitteeName as "CommitteeName", \
    Committees.CommitteeRole as "CommitteeRole" from dse203finalproject.Committees \
    left join dse203finalproject.StateReps on Committees.RepresentativeName = StateReps.LastName \
    where Committees.State is NULL and StateReps.RepresentativeID is NOT NULL \
    UNION \
    Select StateReps.RepresentativeID as "RepresentativeID", Committees.CommitteeName as "CommitteeName", \
    Committees.CommitteeRole as "CommitteeRole" from dse203finalproject.Committees \
    left join dse203finalproject.StateReps on Committees.RepresentativeName = StateReps.LastName \
    and Committees.State = StateReps.Statelabel where Committees.State is NOT NULL and \
    StateReps.RepresentativeID is NOT NULL'
#cursor.execute(querycomm2in)

***BILLS***

In [ ]:
queryxml='CREATE TABLE dse203finalproject.xmlbaseraw (BillID SERIAL PRIMARY KEY NOT NULL, Link varchar(255), OfficialTitle varchar(2500), \
            BillStage varchar(255), LegisType varchar(255))'
#cursor.execute(queryxml)
queryxmldate='CREATE TABLE dse203finalproject.xmlbaserawdate (BillID INT references \
    dse203finalproject.xmlbaseraw(BillID) NOT NULL, Date varchar(255))'
#cursor.execute(queryxmldate)
queryxmlcomm='CREATE TABLE dse203finalproject.xmlbaserawcommittee (BillID INT references \
    dse203finalproject.xmlbaseraw(BillID) NOT NULL, CommitteeName varchar(255))'
#cursor.execute(queryxmlcomm)
queryxmlspon='CREATE TABLE dse203finalproject.xmlbaserawsponsor (BillID INT references \
    dse203finalproject.xmlbaseraw(BillID) NOT NULL, FirstName varchar(255), LastName varchar(255), State varchar(255))'
#cursor.execute(queryxmlspon)
queryxmlcospon='CREATE TABLE dse203finalproject.xmlbaserawcosponsor (BillID INT references \
    dse203finalproject.xmlbaseraw(BillID) NOT NULL, FirstName varchar(255), LastName varchar(255), State varchar(255))'
#cursor.execute(queryxmlcospon)
countind=0
for ind in xmlnames.index:
    link= str("'"+xmlnames.loc[ind, 'Link'].replace("'", '')+"'")
    title=str("'"+xmlnames.loc[ind, 'OfficialTitle'].replace("'", '')+"'")
    stage=str("'"+xmlnames.loc[ind, 'BillStage']+"'")
    legis=str("'"+xmlnames.loc[ind, 'LegisType']+"'")
    queryxinsert="INSERT INTO dse203finalproject.xmlbaseraw   VALUES ("+str(countind)+", "+link+ \
        ", "+title + ", " +stage+ ", " +legis+")" 
    #cursor.execute(queryxinsert)
    if '[' not in str(xmlnames.loc[ind, 'Date']) and str(xmlnames.loc[ind, 'Date'])!='':
        date=str("'"+xmlnames.loc[ind, 'Date'].replace("'", '')+"'")
        queryxinsertd="INSERT INTO dse203finalproject.xmlbaserawdate   VALUES ("+str(countind)+", "+date+")"
        #cursor.execute(queryxinsertd)
    elif str(xmlnames.loc[ind, 'Date'])!='':
        dates=xmlnames.loc[ind, 'Date']
        for dateind in dates:
            date=str("'"+dateind.replace("'", '')+"'")
            queryxinsertd="INSERT INTO dse203finalproject.xmlbaserawdate  VALUES ("+str(countind)+", "+date+")"
            #cursor.execute(queryxinsertd)
    if '[' not in str(xmlnames.loc[ind, 'CommitteeName']) and str(xmlnames.loc[ind, 'CommitteeName'])!='':
        comm=str("'"+xmlnames.loc[ind, 'CommitteeName'].replace("'", '')+"'")
        queryxinsertc="INSERT INTO dse203finalproject.xmlbaserawcommittee   VALUES ("+str(countind)+", "+comm+")"
        #cursor.execute(queryxinsertc)
    elif str(xmlnames.loc[ind, 'CommitteeName'])!='':
        comms=xmlnames.loc[ind, 'CommitteeName']
        for commind in comms:
            comm=str("'"+commind.replace("'", '')+"'")
            queryxinsertc="INSERT INTO dse203finalproject.xmlbaserawcommittee  VALUES ("+str(countind)+", "+comm+")"
            #cursor.execute(queryxinsertc)
    if '[' not in str(xmlnames.loc[ind, 'SponsorLastName']) and str(xmlnames.loc[ind, 'SponsorLastName'])!='':
        queryxinsertssp="INSERT INTO dse203finalproject.xmlbaserawsponsor  VALUES ("+str(countind)+", "+str("'"+xmlnames.loc[ind, 'SponsorFirstName'].replace("'", '')+"'") \
        + ", "+str("'"+xmlnames.loc[ind, 'SponsorLastName'].replace("'", '')+"'")+ ", "+str("'"+xmlnames.loc[ind, 'SponsorState']+"'")+")" 
        #cursor.execute(queryxinsertssp)
    elif str(xmlnames.loc[ind, 'SponsorLastName'])!='':
        lasts=xmlnames.loc[ind, 'SponsorLastName']
        countnames=0
        for lastind in lasts:
            last=str("'"+lastind.replace("'", '')+"'")
            first=str("'"+xmlnames.loc[ind, 'SponsorFirstName'][countnames].replace("'", '')+"'")
            state=str("'"+xmlnames.loc[ind, 'SponsorState'][countnames]+"'")
            queryxinsertssp="INSERT INTO dse203finalproject.xmlbaserawsponsor  VALUES ("+str(countind)+", "+first+", "+last+ ", "+state+")"
            #cursor.execute(queryxinsertssp)
            countnames=countnames+1
    if '[' not in str(xmlnames.loc[ind, 'CoSponsorLastName']) and str(xmlnames.loc[ind, 'CoSponsorLastName'])!='':
        queryxinsertsco="INSERT INTO dse203finalproject.xmlbaserawcosponsor  VALUES ("+str(countind)+", "+str("'"+xmlnames.loc[ind, 'CoSponsorFirstName'].replace("'", '')+"'")+ \
        ", "+str("'"+xmlnames.loc[ind, 'CoSponsorLastName'].replace("'", '')+"'")+ ", "+str("'"+xmlnames.loc[ind, 'CoSponsorState']+"'")+")" 
        #cursor.execute(queryxinsertsco)
    elif str(xmlnames.loc[ind, 'CoSponsorLastName'])!='':
        lasts=xmlnames.loc[ind, 'CoSponsorLastName']
        countnames=0
        for lastind in lasts:
            last=str("'"+lastind.replace("'", '')+"'")
            first=str("'"+xmlnames.loc[ind, 'CoSponsorFirstName'][countnames].replace("'", '')+"'")
            state=str("'"+xmlnames.loc[ind, 'CoSponsorState'][countnames]+"'")
            queryxinsertsco="INSERT INTO dse203finalproject.xmlbaserawcosponsor  VALUES ("+str(countind)+", "+first+", "+last+ ", "+state+")"
            #cursor.execute(queryxinsertsco)
            countnames=countnames+1
        
    countind=countind+1

***BILLS Referencing REPRESENTATIVE ID***

In [ ]:
queryspon2='CREATE TABLE dse203finalproject.SponsorsbyID(BillID INT references \
    dse203finalproject.xmlbaseraw(BillID) NOT NULL, RepresentativeID INT references \
    dse203finalproject.StateReps (RepresentativeID) NOT NULL)'    
#cursor.execute(queryspon2)
querysponin2='INSERT INTO dse203finalproject.SponsorsbyID (BillID, RepresentativeID) \
Select xmlbaserawsponsor.BillID as "BillID", StateReps.RepresentativeID as "RepresentativeID" from dse203finalproject.xmlbaserawsponsor \
    left join dse203finalproject.StateReps on xmlbaserawsponsor.LastName = StateReps.LastName \
    where xmlbaserawsponsor.State is NULL and StateReps.RepresentativeID is NOT NULL \
    UNION \
    Select xmlbaserawsponsor.BillID as "BillID", StateReps.RepresentativeID as "RepresentativeID" from dse203finalproject.xmlbaserawsponsor \
    left join dse203finalproject.StateReps on xmlbaserawsponsor.LastName = StateReps.LastName \
    and xmlbaserawsponsor.State = StateReps.Statelabel where xmlbaserawsponsor.State is NOT NULL and \
    StateReps.RepresentativeID is NOT NULL'
#cursor.execute(querysponin2)
querycspon2='CREATE TABLE dse203finalproject.CosponsorsbyID(BillID INT references \
    dse203finalproject.xmlbaseraw(BillID) NOT NULL, RepresentativeID INT references \
    dse203finalproject.StateReps (RepresentativeID) NOT NULL)'    
#cursor.execute(querycspon2)
querycsponin2='INSERT INTO dse203finalproject.CosponsorsbyID (BillID, RepresentativeID) \
Select xmlbaserawcosponsor.BillID as "BillID", StateReps.RepresentativeID as "RepresentativeID" from dse203finalproject.xmlbaserawcosponsor \
    left join dse203finalproject.StateReps on xmlbaserawcosponsor.LastName = StateReps.LastName \
    where xmlbaserawcosponsor.State is NULL and StateReps.RepresentativeID is NOT NULL \
    UNION \
    Select xmlbaserawcosponsor.BillID as "BillID", StateReps.RepresentativeID as "RepresentativeID" from dse203finalproject.xmlbaserawcosponsor \
    left join dse203finalproject.StateReps on xmlbaserawcosponsor.LastName = StateReps.LastName \
    and xmlbaserawcosponsor.State = StateReps.Statelabel where xmlbaserawcosponsor.State is NOT NULL and \
    StateReps.RepresentativeID is NOT NULL'
#cursor.execute(querycsponin2)